<a href="https://colab.research.google.com/github/mkujaggi/Seq2Seq_Chatbot/blob/master/MJAG7682_COMP5046_Ass1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

# Readme
*If there is something to be noted for the user, please mention here.* 
To run the chat with pre trained model kindly run the following sections:
1.1, 1.2, 2.1, 2.1.6, 2.2.1, 2.2.2,  2.2.3, 2.2.4, 2.2.5, 3.1, 3.2, 3.3, 3.4, 3.5.2
*If you are planning to implement a program with Object Oriented Programming style*

***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing (Personality chat datasets)

## 1.1. Download Dataset (Personality chat datasets)

In [0]:
import json
import re

import tensorflow as tf
import numpy as np
import pandas as pd
import nltk
import tensorflow as tf

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords as sw
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
id = '1ZPLdx8KBBQVSEo760SQ0hoQ85hlN_0Cu'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('qna_chitchat_the_comic.tsv')  
id2 = '1NfALGud5wBmunQ61Km9B3fbYw40VJra5'
downloaded = drive.CreateFile({'id':id2}) 
downloaded.GetContentFile('qna_chitchat_the_friend.tsv')  
id3 = '1nO7DPjhSdRKkptUZNU6dDGOGZhmcFzhy'
downloaded = drive.CreateFile({'id':id3}) 
downloaded.GetContentFile('qna_chitchat_the_professional.tsv')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
    100% |████████████████████████████████| 993kB 10.7MB/s 


## 1.2. Preprocess data (Personality chat datasets)

Using Tweet tokenizer as it does not seperate words with apostrophe and converting the words to lower case as it will help predicting the word.*
*Using lemmatizing because by converting the word to its root form gives result which are in the language and chatbot responses will be more efficient. Also removing the stop words as it will reduce the time in lemmatizing and and stop words are not required in the chat

In [0]:
# Fetching chat personality data
df_professional = pd.read_csv('qna_chitchat_the_professional.tsv', sep="\t")
df_comic = pd.read_csv('qna_chitchat_the_comic.tsv', sep="\t")
df_friend = pd.read_csv('qna_chitchat_the_friend.tsv', sep="\t")
# Class for preprocessing data
class ChatPersonality:
  def preprocessData(self, df):
    unique_words=[]
    whole_data = []
    chat_data = []
    max_input_word = 0
    for index, row in df.iterrows():
      question = row.Question
      answer = row.Answer
      chat_data.append([question,answer])
      question = re.sub(r'[^\w\s]','',question) #Removing Special characters
      question = question.lower()
      q_token = tknzr.tokenize(question) #Tweet Tokenizing the data
      a_token = [answer]
      q_lemmatized = []
      for w in q_token:
        q_lemmatized.append(wordnet_lemmatizer.lemmatize(w)) # Lemmatizing data
      whole_data += q_lemmatized
      whole_data += a_token
      max_input_word = max(len(q_lemmatized), max_input_word)
    unique_words = list(set(whole_data))
    unique_words.append('_B_')
    unique_words.append('_E_')
    unique_words.append('_P_')
    unique_words.append('_U_')
    num_dic = {n: i for i, n in enumerate(unique_words)}
    dic_len = len(num_dic)
    return num_dic, max_input_word, chat_data, unique_words
obj = ChatPersonality()

dic_professional, max_input_professional, chat_professional, unique_words_pro = obj.preprocessData(df_professional)
dic_comic, max_input_comic, chat_comic, unique_words_com = obj.preprocessData(df_comic)
dic_friend, max_input_friend, chat_friend, unique_words_fri = obj.preprocessData(df_friend)

# 2 - Model Implementation

## 2.1. Word Embeddings

Using  fastText as it will be able to predict more number of words than in the dataset. Using CBOW as it will see the context of the one word and not give out multiple words as in as in Skip-gram, also since we are dealing with large data set using CBOW would be a better approach. 

In [0]:
!pip install paramiko
from gensim.models import FastText

    100% |████████████████████████████████| 194kB 6.7MB/s 
    100% |████████████████████████████████| 768kB 24.9MB/s 
    100% |████████████████████████████████| 61kB 24.3MB/s 
    100% |████████████████████████████████| 2.3MB 11.2MB/s 
    100% |████████████████████████████████| 102kB 30.7MB/s 


### 2.1.1. Download Dataset for Word Embeddings



*Choosing movie lines dataset because it contains large vocabolary for model to better train and also, since it contains movie lines it would help train the model with regular slangs as well. *


In [0]:
#Downloading Movie lines dataset from Google drive
id4 = '1bTKKJemxya3QHSCGJWuyf3UeP5UGxnmg'
downloaded = drive.CreateFile({'id':id4}) 
downloaded.GetContentFile('movie_lines.txt')

### 2.1.2. Data Preprocessing for Word Embeddings

Using Tweet tokenizer as it does not seperate words with apostrophe and converting the words to lower case as it will help predicting the word. Not removing stop words as it will loose the context to certain words and using lemmatizing because by converting the word to its root form gives result which are in the language and chatbot responses will be more efficient. Also removing the  special characters to reduce the vocabolary and clean the text.

In [0]:
# Preprocessing data for Word Embeddings
df_movie_lines = []
df_movie = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
for line in df_movie:
  #extracting dialogues from data file
  lines = line.split(' +++$+++ ')
  m_line = re.sub(r'[^\w\s]','',lines[-1])
  m_line = m_line.lower()
  # Tweet tokenizing the data
  m_line_token = tknzr.tokenize(m_line)
  m_lemma = []
  for line in m_line_token:
    m_lemma.append(wordnet_lemmatizer.lemmatize(line))
  df_movie_lines.append(m_lemma)

### 2.1.3. Build Word Embeddings Model

Choosing window size as 3 since dataset contains small conversations, so taking a small window size will help finding the context of the word, as it will take 3 words from either side of the center word. Also taking size of 100  at a time.  Taking Min_count as five to remove the rare words from the vocab.

In [0]:
# Please comment your code
ft_cbow_model = FastText(size=100, window=3, min_count=5, workers=4, sg=0)
ft_cbow_model.build_vocab(sentences=df_movie_lines)

### 2.1.4. Train Word Embeddings Model

In [0]:
# Training word Embeddings and Printing the Loss

ft_cbow_model.train(sentences=df_movie_lines, total_examples=len(df_movie_lines), epochs=100)

### 2.1.5. Save Word Embeddings Model

In [0]:
# Saving word embeddings in the google drive
from google.colab import drive
drive.mount("/content/gdrive")

ft_cbow_model.save('/content/gdrive/My Drive/assignment1/movie_dialog_title.model')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 2.1.6. Load Word Embeddings Model

In [0]:
# Retriving world embeddings from Google Drive
from google.colab import drive
drive.mount("/content/gdrive")
cow_model = FastText.load("/content/gdrive/My Drive/assignment1/movie_dialog_title.model")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## 2.2. Seq2Seq model

### 2.2.1. Apply/Import Word Embedding Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
# Converting answers data to vectors format
def get_vectors_a(sentence, dic_personality):    
    tokenized_sentence = [sentence]
    data = tokens_to_ids(tokenized_sentence, dic_personality)
    return data

def tokens_to_ids(tokenized_sentence, dic_personality):
    ids = []
    for token in tokenized_sentence:
        if token in dic_personality:
            ids.append(dic_personality[token])
        else:
            ids.append(dic_personality['_U_'])   

    return ids
# Method for converting Personality dataset to vectors for output of Tensorflow 
def make_batch(seq_data, dic_personality):
    output_batch = []
    target_batch = []

    for seq in seq_data:              
        # Input for decoder cell, Add '_B_' at the beginning of the sequence data
        output_data = [dic_personality['_B_']]
        output_data += get_vectors_a(seq[1], dic_personality)
        # Output of decoder cell, Add '_E_' at the end of the sequence data
        target = get_vectors_a(seq[1], dic_personality)
        target.append(dic_personality['_E_'])
        
        # Convert each token vector to one-hot encode data
        output_batch.append(np.eye(len(dic_personality))[output_data])
        
        target_batch.append(target)
    
    return output_batch, target_batch
# Method for creating vectors from WordEmbeddings for input in Tensorflow
def get_input_vectors(model):
  id_to_token = ['_U_']
  token_to_id = {'_U_': 0}
  for tokens in model.wv.vocab:
    if tokens not in token_to_id:
        token_to_id[tokens] = len(token_to_id)
        id_to_token.append(tokens)
  vector_list = []
  scale = np.sqrt(3.0 / 100)
  for word in id_to_token:
    if word in model.wv.vocab:
      vector_list.append(np.array(model.wv[word]))
    else:
      random_vector = np.random.uniform(-scale, scale, [100])
      vector_list.append(random_vector)
  return vector_list
vector_list = get_input_vectors(cow_model)

### 2.2.2. Build Seq2Seq Model

Choosing a learning rate and epoch that doe not take a lot of time and does not cause instability.


In [0]:
learning_rate = 0.01
n_hidden = 128


def encoder():
  tf.reset_default_graph()
  enc_input = tf.placeholder(tf.float32, shape=[None,None,100])
   # Encoder Cell
  with tf.variable_scope('encode'):
      enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
      enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

      outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                              dtype=tf.float32)
  return enc_input

def decoder(dic_personality):
  n_class = len(dic_personality)
  n_input = len(dic_personality)
  # encoder/decoder shape = [batch size, time steps, input size]
  dec_input = tf.placeholder(tf.float32, [None, None, n_input])
  
  # target shape = [batch size, time steps]
  targets = tf.placeholder(tf.int64, [None, None])

  # Decoder Cell
  with tf.variable_scope('decode'):
      dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
      dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

      outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                              dtype=tf.float32)
  
  model = tf.layers.dense(outputs, n_class, activation=None)

  cost = tf.reduce_mean(
              tf.nn.sparse_softmax_cross_entropy_with_logits(
                  logits=model, labels=targets))

  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
  return dec_input, targets, cost, optimizer, model



### 2.2.3. Train Seq2Seq Model

In [0]:
# Please comment your code
def trainModel(optimizer, cost, enc_input, dec_input, targets, vector_list, output_batch, target_batch):
  total_epoch = 5000
  for epoch in range(total_epoch):
      _, loss = sess.run([optimizer, cost],
                         feed_dict={enc_input: [vector_list],
                                    dec_input: output_batch,
                                    targets: target_batch})
      if epoch % 100 == 0:
          print('Epoch:', '%04d' % (epoch + 1),
                'cost =', '{:.6f}'.format(loss))

  print('Epoch:', '%04d' % (epoch + 1),
        'cost =', '{:.6f}'.format(loss))
  print('Training completed')


### 2.2.4. Save Seq2Seq Model

In [0]:
# Please comment your code
from google.colab import drive
drive.mount("/content/gdrive")
def train_and_save(personality):
  
  saver = tf.train.Saver()
  save_path = saver.save(sess, "/content/gdrive/My Drive/assignment1/SS_model_"+personality+".ckpt")
  print("Model saved in path: %s" % save_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 2.2.5. Load Seq2Seq Model

In [0]:
# Loading trained data from google drive
def loadModel(personality):
  pro = tf.Graph()
  personality = personality.lower()
  sess = tf.Session(graph = pro)
  with pro.as_default():
    saver = tf.train.import_meta_graph("/content/gdrive/My Drive/assignment1/SS_model_"+personality+".ckpt.meta")

    sess.run(tf.global_variables_initializer())

    saver.restore(sess, "/content/gdrive/My Drive/assignment1/SS_model_"+personality+".ckpt")

# 3 - Evaluation (Running chatbot)

## 3.1. Start chatting

In [0]:
dic_personality = dic_professional
person="professional"
max_input_personality = max_input_professional
unique_words = unique_words_pro
def answer(sentence):

  global count
  global max_input_personality
  global dic_personality
  global person
  global unique_words
  if sentence[:7] =="*change":
    dic_personality, max_input_personality, tmp_person, unique_words = changePersonality(sentence, True)      
    if max_input_personality == 0:
      dic_personality, max_input_personality, person, unique_words = changePersonality("*change: "+person, True)
      return "Personality not found. Kindly choose from friend, comic or professional."
    elif tmp_person == "friend":
      person = tmp_person
      enc_input = encoder()
      dec_input, targets, cost, optimizer, model = decoder(dic_personality)
      return "Now I will talk like a friend."
    elif tmp_person == "comic":
      person = tmp_person
      enc_input = encoder()
      dec_input, targets, cost, optimizer, model = decoder(dic_personality)
      return "Now I will talk like a comic"
    elif tmp_person == "professional":
      person = tmp_person
      enc_input = encoder()
      dec_input, targets, cost, optimizer, model = decoder(dic_personality)
      return "Now I will talk like a professional."
  elif  sentence[:4].lower() == "quit":
    return -1
  else:
    enc_input = encoder()
    dec_input, targets, cost, optimizer, model = decoder(dic_personality)
  seq_data = [sentence, '_U_' * 1]
  sess = tf.Session()
  sess.run(tf.global_variables_initializer())
  output_batch, target_batch = make_batch([seq_data], dic_personality)
  prediction = tf.argmax(model, 2)
  result = sess.run(prediction,
                    feed_dict={enc_input: [vector_list],
                               dec_input: output_batch,
                               targets: target_batch})

  # convert index number to actual token 
  decoded = [unique_words[i] for i in result[0]]
  # Remove anything after '_E_'        
  if "_E_" in decoded:
      end = decoded.index('_E_')
      translated = ' '.join(decoded[:end])
  elif "_B_" in decoded:
    start = decoded.index('_B_')
    translated = ' '.join(decoded[start:])
  else :
      translated = ' '.join(decoded[:])

  return translated

## 3.2. Change Personality

*To change the personality in the chat type "*change: personality" 
Example type "\*change: comic"

In [0]:
# Please comment your code
def changePersonality(sentence, isChange):
  if isChange:
    dic_personality = {}
    max_input_personality = 0
    unique_words = []
    personality = re.split(r':', sentence)
    person = personality[-1].strip()
    if person.lower() == "comic":
      dic_personality = dic_comic
      max_input_personality = max_input_comic
      unique_words = unique_words_com
      loadModel(person)
    elif person.lower() == "friend":
      dic_personality = dic_friend
      max_input_personality = max_input_friend
      unique_words = unique_words_fri
      loadModel(person)
    elif person.lower() == "professional":
      dic_personality = dic_professional
      max_input_personality = max_input_professional
      unique_words = unique_words_pro
      loadModel(person)
    else:
      return dic_personality, max_input_personality, person.lower(), unique_words
  else:
    dic_personality = dic_professional
    max_input_personality = max_input_professional
    unique_words = unique_words_pro
    person = "professional"
  return dic_personality, max_input_personality, person.lower(), unique_words
  


## 3.3. Save chat log

In [0]:
# Please comment your code
def save_chat_log(chat_log):
  chatLogFile = open('/content/gdrive/My Drive/assignment1/MJAG7682_chat_log.txt', 'a')
  for chat in chat_log:
    chatLogFile.write("\n"+str(chat))
  chatLogFile.write('\n'+'---------------------------------------------------')
  chatLogFile.close()  

## 3.4. End chatting

To end chat just type "quit" in the chat.

In [0]:
# Please comment your code
def end_chatting(chat_log):
  save_chat_log(chat_log)

## 3.5. Execute program

***Please make sure your program  is running properly.***

***Functions for downloading (from Google Drive) and loading models (both word embeddings and Seq2Seq) need to be called!*** 


### 3.5.1. Execute program - training mode

*Please include lines to train the bot.*

In [0]:
output_batch, target_batch = make_batch(chat_comic ,dic_comic)
enc_input= encoder()
dec_input, targets, cost, optimizer, model = decoder(dic_comic)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
trainModel(optimizer, cost, enc_input, dec_input, targets, vector_list, output_batch, target_batch)
train_and_save('comic')


enc_input= encoder()
output_batch, target_batch = make_batch(chat_professional ,dic_professional)

dec_input, targets, cost, optimizer, model = decoder(dic_professional)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
trainModel(optimizer, cost, enc_input, dec_input, targets, vector_list, output_batch, target_batch)
train_and_save('professional')

output_batch, target_batch = make_batch(chat_friend ,dic_friend)
enc_input= encoder()
dec_input, targets, cost, optimizer, model = decoder(dic_friend)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
trainModel(optimizer, cost, enc_input, dec_input, targets, vector_list, output_batch, target_batch)

train_and_save('friend')

Epoch: 0001 cost = 6.411785
Epoch: 0101 cost = 2.243492
Epoch: 0201 cost = 2.247497
Epoch: 0301 cost = 2.247127
Epoch: 0401 cost = 2.233783
Epoch: 0501 cost = 2.237411
Epoch: 0601 cost = 2.234044
Epoch: 0701 cost = 2.239955
Epoch: 0801 cost = 2.240420
Epoch: 0901 cost = 2.238179
Epoch: 1001 cost = 2.243143
Epoch: 1101 cost = 2.243031
Epoch: 1201 cost = 2.234694
Epoch: 1301 cost = 2.236718
Epoch: 1401 cost = 2.233294
Epoch: 1501 cost = 2.239013
Epoch: 1601 cost = 2.239004
Epoch: 1701 cost = 2.235436
Epoch: 1801 cost = 2.236678
Epoch: 1901 cost = 2.236172
Epoch: 2001 cost = 2.234873
Epoch: 2101 cost = 2.239042
Epoch: 2201 cost = 2.236275
Epoch: 2301 cost = 2.234096
Epoch: 2401 cost = 2.238358
Epoch: 2501 cost = 2.236131
Epoch: 2601 cost = 2.232791
Epoch: 2701 cost = 2.239544
Epoch: 2801 cost = 2.236048
Epoch: 2901 cost = 2.236382
Epoch: 3001 cost = 2.232912
Epoch: 3101 cost = 2.231715
Epoch: 3201 cost = 2.234505
Epoch: 3301 cost = 2.235190
Epoch: 3401 cost = 2.229778
Epoch: 3501 cost = 2

### 3.5.2. Execute program - chatting mode

*Please include lines to start chatting with the bot.*

In [0]:
# Please comment your code
loadModel("professional")
def chatBot():
  Continue = 1
  chatlog = []
  
  print("Bot: Hello")
  count = 0
  chatlog.append("Bot: Hello")
  while Continue == 1:
    
    ques = input("You: ")
    
    result = answer(ques)
    count = count+1
    chatlog.append("You: "+ques)
    if result == -1:
      print("Bot: Chat saved.See you later.")
      chatlog.append("Bot: Chat saved.See you later.")
      end_chatting(chatlog)
      Continue = 0
    else:
      print("Bot: "+result)
      chatlog.append("Bot: "+result)
chatBot()

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/assignment1/SS_model_professional.ckpt
Bot: Hello
You: HI
Bot: worst alligator
You: yes aligators are the worst
Bot: google job
You: you can also do a google job
Bot: season interest
You: So how are you?
Bot: ask get
You: What is your favorite food?
Bot: greeting I don't need to eat.
You: but why?
Bot: I'm here to answer your questions and help out. mad
You: So you must get hungry while answering so many questions
Bot: feel afraid
You: Don't be afraid
Bot: tomorrow your
You: ok cool
Bot: guy tall
You: Yes a guy can be tall
Bot: Very well. why
You: ok are you tall?
Bot: yup I enjoy talking with you.
You: Thank you. I also enjoy talking to you
Bot: not whered
You: ok
Bot: getting and
You: greetings
Bot: thanks sibling
You: your welcome
Bot: afraid favorite
You: time so say good bye
Bot: at yesterday
You: *change: comic
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/assignment1/SS_model_comic.ckpt
Bot: 

## Object Oriented Programming codes here

*If you have multiple classes use multiple code snippets to add them.*

In [0]:
# If you used OOP style, use this sectioon